# RHF 自洽场相关基础

这一节开始，我们将会接触实际的量化程序．我们从自洽场的计算出发，初步了解公式记号、积分和基组调用、张量计算与 PySCF 函数调用．

In [ ]:
import numpy as np
import scipy
from pyscf import scf, gto, lib
from functools import partial

import sys
sys.path.append('../../src')
from utilities import val_from_fchk

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(5, linewidth=150, suppress=True)

<div class="alert alert-info">

**任务**

1. 上面的代码块是引入外部程序以及作文档初始化的代码块．请解释上述代码的每一行的意义．

   不同的文档会有不同的初始化代码块，即使这些代码块可能看起来一样．请在阅读一份新的文档之前检查第一个代码块与其它文档是否有不同．

</div>

## PySCF 自洽场计算

我们先作一个 PySCF 的自洽场计算．这将是我们使用 PySCF 的第一个任务．我们以后会一直使用下面的 $C_1$ 对称的双氧水分子作为范例：

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! NOTE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# For clear and simplicity, following code cell will be hidden and assumed to be executed!

mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

而 PySCF 的 RHF 自洽场可以通过下述代码实现：

In [ ]:
scf_eng = scf.RHF(mol)
scf_eng.conv_tol = 1e-12
scf_eng.conv_tol_grad = 1e-10
scf_eng.kernel()

我们也可以使用 Gaussian 计算得到相同的结果 ([输入文件](include/basic_scf/rhf_energy.gjf))：

In [ ]:
np.allclose(
    val_from_fchk("SCF Energy", "include/basic_scf/rhf_energy.fchk"),
    scf_eng.e_tot
)

## 小型自洽场程序

我们现在就根据 Szabo and Ostlund <cite data-cite="Szabo-Ostlund.Dover.1996"></cite> 书籍上的指示 (p146) 进行最简单的 SCF 程序编写．

<div class="alert alert-info">

**提醒**

由于该分子已经难以通过零密度初猜来得到能量，因此这里暂且利用了 PySCF 所提供的初猜．

</div>

In [ ]:
S = mol.intor("int1e_ovlp")
HC = mol.intor("int1e_kin") + mol.intor("int1e_nuc")
eri = mol.intor("int2e")
X = scipy.linalg.fractional_matrix_power(S, -0.5)

natm = mol.natm
nmo = nao = mol.nao
nocc = mol.nelec[0]
so = slice(0, nocc)

<div class="alert alert-info">

**任务**

1. (可选) 我们在上面的代码块中已经生成了电子积分，随后我们的工作仅仅是给出一个 SCF 算法．这无外乎 Python 与 numpy 的代码书写．你完全可以尝试不看下面的代码，自己先试写一个 SCF 过程．这可以是研究生一年级的量化程序大作业．你可能必须要一个更好的初猜；零密度初猜似乎对双氧水分子不适用．密度矩阵的初猜可以通过以下代码得到

   ```python
   D = scf_eng.get_init_guess()
   ```

   如果不清楚上面的代码块的变量意义，可以参考 [后面的文档](#sixth_code_block)．
   
   提示：你可以了解 [np.linalg.eigh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eigh.html) 函数的意义，并思考它可能对 SCF 过程的编写有何帮助．
   
2. (可选) 体系总能量包括电子能量 $E_\mathrm{elec}$ 与原子核排斥能 $E_\mathrm{nuc}$；前者通过 SCF 过程获得．你可以尝试不看下面的代码，先自己写一个原子核排斥能的计算过程．你可能会使用到 `gto.Mole.atom_coords` 函数与 `mol.Mole.atom_charges` 函数，但你也可以不使用它们．

   进阶：尝试完全不使用 Python 的 `for` 语句构建 $E_\mathrm{nuc}$．

   提示：我们所有的计算都应当在 a.u. 单位下，但我们输入的原子坐标单位是 Angstrom．请检查计算过程中，单位是否正确．PySCF 中 Bohr 半径常量定义可以通过 `lib.param.BOHR` 获得．

</div>

In [ ]:
A_dist = np.diag(np.ones(natm) * np.inf) + np.linalg.norm(
    mol.atom_coords()[:, None, :] - mol.atom_coords()[None, :, :],
    axis=2)
A_charge = mol.atom_charges()[:, None] * mol.atom_charges()
E_nuc = 0.5 * (A_charge / A_dist).sum()
print("Neucleus energy   ", E_nuc, " a.u.")

In [ ]:
D = scf_eng.get_init_guess()
D_old = np.zeros((nao, nao))
count = 0

while (not np.allclose(D, D_old)):
    if count > 500:
        raise ValueError("SCF not converged!")
    count += 1
    D_old = D
    F = HC + np.einsum("uvkl, kl -> uv", eri, D) - 0.5 * np.einsum("ukvl, kl -> uv", eri, D)
    Fp = X.T @ F @ X
    e, Cp = np.linalg.eigh(Fp)
    C = X @ Cp
    D = 2 * C[:, so] @ C[:, so].T

E_elec = (HC * D).sum() + 0.5 * np.einsum("uvkl, uv, kl ->", eri, D, D) - 0.25 * np.einsum("ukvl, uv, kl ->", eri, D, D)
E_tot = E_elec + E_nuc

print("SCF Converged in  ", count, " loops")
print("Electronic energy ", E_elec, " a.u.")
print("Total energy      ", E_tot, " a.u.")
print("----------------- ")
print("Energy allclose   ", np.allclose(E_tot, scf_eng.e_tot))
print("Density allclose  ", np.allclose(D, scf_eng.make_rdm1()))

## 代码说明

我们一点一点地对代码进行说明．

第五个代码块中，我们定义了三个电子积分、$X_{\mu \nu}$ 矩阵以及与维度有关的量．除去分子轨道数，其余都是只与分子和基组有关的量．而一般来说，只要没有原子轨道线性依赖的情况，一般的程序都会定义分子轨道数与原子轨道基组数一致．

* `S`，或 `int1e_ovlp` 指交换积分，其在 Szabo (3.136) 定义，其表达式为 $S_{\mu \nu}$

* `HC` 为动能积分 `int1e_kin` 与核排斥势积分 `int1e_nuc` 的和，其在 Szabo (3.149) 定义，其表达式为 $h_{\mu \nu}$

* `eri`，或 `int2e` 指双电子互斥积分，其在 Szabo Table 2.2 定义，其表达式为 $(\mu \nu | \kappa \lambda)$，采用 Chemistry Convention

* `natm` 为原子数

* `nmo` 为分子轨道数，以后默认与原子轨道数相等，但一般地，根据表达式总是可以区分我们应该采用原子轨道还是分子轨道．

* `nao` 为原子轨道数

* `nocc` 为占据轨道数；以后会出现 `nvir`，为非占轨道数

* `X` 只在自洽场过程中出现，以后将不使用．其表达式为 $X_{\mu \nu}$，并满足关系式 Szabo (3.165) $X_{\mu \kappa} S_{\kappa \lambda} X_{\nu \lambda} = \delta_{\mu \nu}$

<div class="alert alert-info">

**记号说明**

* $\mu, \nu, \kappa, \lambda$ 代表原子轨道

* $i, j, k, l$ 代表分子轨道，但出于程序编写需要 $k, l$ 尽量不能与 $\kappa, \lambda$ 同时出现

* $a, b, c, d$ 代表非据轨道

* $p, q, r, s$ 代表全分子轨道，但 $r, s$ 的使用需要尽量避免

* $t, s, r, w$ 代表坐标分量；一般 $t, s$ 特指原子坐标分量，$r, w$ 特指电子坐标分量；坐标分量的三种可能取向是 $x, y, z$ 方向

* $A, B, M$ 代表原子；其中 $M$ 一般是被求和的角标

* $\boldsymbol{A}, \boldsymbol{B}, \boldsymbol{M}$ 代表原子坐标，区别于普通斜体字母；也作为三维向量，区别于 $t, s$

</div>

In [ ]:
np.allclose(X @ S @ X.T, np.eye(nao))

<a id='sixth_code_block'></a>

第六个代码块中，我们计算了核排斥能．

* `A_dist` 表示原子之间的欧氏距离，其中的对角元设为无穷大，是为了让 $R_{MM}^{-1}$ 定义为零

  $$
  R_{AB} = |\boldsymbol{A} - \boldsymbol{B}| = \sqrt{\sum_{t} (A_t - B_t)^2}
  $$

* `A_charge` 表示两原子的电荷乘积

  $$
  Z_{AB} = Z_A Z_B
  $$

* `E_nuc` 为原子核排斥能，以 a.u. 为单位：

  $$
  E_\mathrm{nuc} = \frac{1}{2} Z_{AB} R_{AB}^{-1}
  $$

第七个代码块是具体执行 SCF 计算的代码．

* 第 1 行
  
  ```python
  D = scf_eng.get_init_guess()
  ```
  
  是除了电子积分外唯一使用 PySCF 的代码，它给一个合理的初猜．

* 第 10 行

  ```python
  F = HC + np.einsum("uvkl, kl -> uv", eri, D) - 0.5 * np.einsum("ukvl, kl -> uv", eri, D)
  ```
  
  定义了 Fock 矩阵
  
  $$
  F_{\mu \nu} = h_{\mu \nu} + (\mu \nu | \kappa \lambda) D_{\kappa \lambda} - \frac{1}{2} (\mu  \lambda| \kappa \nu) D_{\kappa \lambda}
  $$
  
* 第 14 行

  ```python
  D = 2 * C[:, so] @ C[:, so].T
  ```
  
  通过使用占据轨道分割 `so`，更新了密度矩阵
  
  $$
  D_{\mu \nu} = 2 C_{\mu i} C_{\nu i}
  $$
  
* 第 16 行

  ```python
  E_elec = (HC * D).sum() + 0.5 * np.einsum("uvkl, uv, kl ->", eri, D, D) - 0.25 * np.einsum("ukvl, uv, kl ->", eri, D, D)
  ```
  
  使用 SCF 收敛后的密度计算总能量
  
  $$
  E_\mathrm{elec} = h_{\mu \nu} D_{\mu \nu} + \frac{1}{2} D_{\mu \nu} (\mu \nu | \kappa \lambda) D_{\kappa \lambda} - \frac{1}{4} D_{\mu \nu} (\mu  \lambda| \kappa \nu) D_{\kappa \lambda}
  $$

## Hamiltonian Core 积分详述

我们刚才提到，在 $h_{\mu \nu}$ 中，有动能的贡献量 $t_{\mu \nu} = \langle \mu | \hat t | \nu \rangle$ 与核排斥能的贡献量 $v_{\mu \nu} = \langle \mu | \hat v_\mathrm{nuc} | \nu \rangle$．这两种积分可以通过更为底层的方式获得；特别是对于核排斥能的贡献量的理解，将会直接地影响到以后对 Hamiltonian Core 的原子核坐标梯度、二阶梯度的理解．

### 动能积分

动能积分可以写为

$$
t_{\mu \nu} = \langle \mu | \hat t | \nu \rangle = - \frac{1}{2} \phi_\mu \cdot (\partial_r^2 \phi_\nu) = - \frac{1}{2} \phi_\mu \phi_{r r \nu}
$$

<div class="alert alert-info">

**记号说明**

* $\phi$ 统一代表原子轨道函数，以电子坐标为自变量

* $\phi_\mu$ 代表原子轨道 $\mu$ 所对应的原子轨道函数

* $\phi_{r \mu} = \partial_r \phi_\mu$ 代表原子轨道在电子坐标分量 $r$ 下的偏导数

* $\phi_{r w \mu} = \partial_r \partial_w \phi_\mu$ 代表原子轨道在电子坐标分量 $r$ 与 $w$ 下的二阶偏导数

* $\boldsymbol{r}$ 作为加粗的 r 代表电子坐标；区别于电子坐标分量 $r$ 是一维变量，$\boldsymbol{r}$ 为三维向量

一般来说，如果一个表达式看起来是函数表达式，那么我们默认对其进行积分．譬如上式若不使用 Einstein Summation，则表达结果是是

$$
t_{\mu \nu} = - \frac{1}{2} \int \phi_{\mu} (\boldsymbol{r}) \nabla_{\boldsymbol{r}}^2 \phi_{\nu} (\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
$$

</div>

在 PySCF 的积分引擎中，一个积分选项是生成关于 $(r, w, \mu, \nu)$ 的 AO 积分张量 $\langle \partial_r \partial_w \mu | \nu \rangle = \phi_{r w \mu} \phi_\nu$ (用变量 `int1e_ipipovlp` 表示)；我们可以对上述张量在 $w = r$ 的情形求和，转置 $\mu, \nu$，并乘以系数 $-0.5$，就得到了动能积分 $t_{\mu \nu}$ 了：

In [ ]:
int1e_ipipovlp = mol.intor("int1e_ipipovlp").reshape((3, 3, nao, nao))
np.allclose(
    - 0.5 * (int1e_ipipovlp.diagonal(axis1=0, axis2=1).sum(axis=2)).T,
    mol.intor("int1e_kin"))

<div class="alert alert-info">

**任务**

1. 上述代码块中使用的是 `sum(axis=2)`，为什么？使用 `sum(axis=0)` 是否正确？

2. 我们还可以用另一种方法生成动能积分．现定义 `int1e_ipovlpip` 为 $\langle \partial_r \mu | \partial_w \nu \rangle$，请解释下述代码块为何输出 True？

   提示 1：考察算符 $\partial_r$ 的性质．
   
   提示 2：动能算符为何是厄米算符？
   
   对这些问题的了解将会允许我们更清晰地理解 AO 积分的对称性，辅助验证程序与公式的正确性，并辅助我们推导核排斥势的导数．

</div>

In [ ]:
int1e_ipovlpip = mol.intor("int1e_ipovlpip").reshape((3, 3, nao, nao))
np.allclose(
    0.5 * (int1e_ipovlpip.diagonal(axis1=0, axis2=1).sum(axis=2)),
    mol.intor("int1e_kin"))

### 核排斥势积分

势能积分可以写为

$$
v_{\mu \nu} = \langle \mu | \frac{- Z_M}{|\boldsymbol{r} - \boldsymbol{M}|} | \nu \rangle
= \langle \mu | \frac{- Z_M}{|\boldsymbol{r}|} | \nu \rangle_{\boldsymbol{r} \rightarrow M}
= \left( \frac{- Z_M}{|\boldsymbol{r}|} \phi_\mu \phi_\nu \right)_{\boldsymbol{r} \rightarrow M}
$$

<div class="alert alert-info">

**记号说明**

* 下标 $\boldsymbol{r} \rightarrow M$ 代表电子积分的原点取在原子 $M$ 的坐标上．

</div>

在 PySCF 的积分引擎中，$\langle \mu | \frac{1}{|\boldsymbol{r}|} | \nu \rangle$ 的积分选项是 `int1e_rinv`；但其积分原点仍然是 $(0, 0, 0)$．为了让特定原子坐标成为原点，PySCF 的一个便利函数是 `gto.Mole.with_rinv_as_nucleus`；它通过传入原子序号，将积分时的原点坐标更变为当前原子的坐标．

In [ ]:
v = np.zeros((nao, nao))
for M in range(natm):
    with mol.with_rinv_as_nucleus(M):
        v += - mol.atom_charge(M) * mol.intor("int1e_rinv")
np.allclose(v, mol.intor("int1e_nuc"))

## 实现参考

在这里以及以后，实现参考一节将会展示不同的实现手段；这可能包括使用 PySCF 的高级函数，或者使用我们手写的 Python 脚本；并与当前的计算结果进行对照．

这份笔记的初衷有二：其一是记录非自洽 DFT 的计算方式；其二是尽可能只使用 PySCF 的积分、泛函与基组库，但不使用高级函数来构建我们的工作．

<div class="alert alert-info">

**提示**

一些不太容易编写，或者与效率有很强关联的程序，我们可能只叙述其原理，但最终还是会使用 PySCF 的库函数．SCF 过程和导出量、双电子积分函数、以及 `pyscf.scf.cphf.solve` 函数将会是其中几个例子．

</div>

### Hamiltonian Core 积分 $h_{\mu \nu}$

In [ ]:
scf_eng.get_hcore.__func__

In [ ]:
np.allclose(
    scf_eng.get_hcore(),
    mol.intor("int1e_kin") + mol.intor("int1e_nuc")
)

In [ ]:
np.allclose(
    scf_eng.get_hcore(),
    HC
)

### 库伦积分 $J_{\mu \nu}[X_{\kappa \lambda}]$

$J_{\mu \nu}[X_{\kappa \lambda}] = (\mu \nu | \kappa \lambda) X_{\kappa \lambda}$

In [ ]:
scf_eng.get_j.__func__

In [ ]:
X = np.random.random((nao, nao))
np.allclose(
    scf_eng.get_j(dm=X),
    np.einsum("uvkl, kl -> uv", mol.intor("int2e"), X)
)

### 交换积分 $K_{\mu \nu}[X_{\kappa \lambda}]$

$K_{\mu \nu}[X_{\kappa \lambda}] = (\mu \kappa | \nu \lambda) X_{\kappa \lambda}$

<div class="alert alert-warning">

**注意**

交换积分对代入的 AO 矩阵有对称性要求．一般来说，我们以后工作中碰到的 AO 矩阵都是对称矩阵，因此 `hermi` 选项可以不设置．

</div>

In [ ]:
scf_eng.get_k.__func__

In [ ]:
X = np.random.random((nao, nao))
[np.allclose(
    scf_eng.get_k(dm=X, hermi=hermi),
    np.einsum("ukvl, kl -> uv", mol.intor("int2e"), X)
) for hermi in [0, 1]]

In [ ]:
X = np.random.random((nao, nao))
X += X.T
[np.allclose(
    scf_eng.get_k(dm=X, hermi=hermi),
    np.einsum("ukvl, kl -> uv", mol.intor("int2e"), X)
) for hermi in [0, 1]]

### Fock 矩阵 $F_{\mu \nu}[X_{\kappa \lambda}]$

$F_{\mu \nu}[X_{\kappa \lambda}] = h_{\mu \nu} + J_{\mu \nu}[X_{\kappa \lambda}] - \frac{1}{2} K_{\mu \nu}[X_{\kappa \lambda}]$

In [ ]:
scf_eng.get_fock.__func__

<div class="alert alert-warning">

**注意**

Fock 矩阵同样对代入的 AO 矩阵有对称性要求．一般来说，我们也只处理对称矩阵．

</div>

In [ ]:
scf_eng.get_fock.__func__

In [ ]:
X = np.random.random((nao, nao))
X += X.T
np.allclose(
    scf_eng.get_fock(dm=X),
    scf_eng.get_hcore() + scf_eng.get_j(dm=X) - 0.5 * scf_eng.get_k(dm=X)
)

### 原子核排斥能 $E_\mathrm{nuc}$

In [ ]:
scf_eng.energy_nuc.__func__

In [ ]:
np.allclose(
    scf_eng.energy_nuc(),
    E_nuc
)

### 体系总能量 $E_\mathrm{elec}[X_{\mu \nu}]$

$E_\mathrm{elec}[X_{\mu \nu}] = (h_{\mu \nu} + \frac{1}{2} J_{\mu \nu} [X_{\kappa \lambda}] - \frac{1}{4} [X_{\kappa \lambda}])  X_{\mu \nu}$

In [ ]:
scf_eng.energy_elec.__func__

`pyscf.scf.hf.energy_elec` 一般有两个返回值，前者是体系总能量；而后者是双电子积分能量：

In [ ]:
X = np.random.random((nao, nao))
X += X.T
print(np.allclose(
    scf_eng.energy_elec(dm=X)[0],
    ((scf_eng.get_hcore() + 0.5 * scf_eng.get_j(dm=X) - 0.25 * scf_eng.get_k(dm=X)) * X).sum()
))
print(np.allclose(
    scf_eng.energy_elec(dm=X)[1],
    ((0.5 * scf_eng.get_j(dm=X) - 0.25 * scf_eng.get_k(dm=X)) * X).sum()
))

### 系数矩阵 $C_{\mu p}$

In [ ]:
type(scf_eng.mo_coeff)

In [ ]:
np.allclose(scf_eng.mo_coeff, C, atol=1e-5, rtol=1e-3)

In [ ]:
np.allclose(abs(scf_eng.mo_coeff / C), np.ones((nao, nmo)), atol=1e-5, rtol=1e-3)

### 密度矩阵 $D_{\mu \nu}$

$D_{\mu \nu} = 2 C_{\mu i} C_{\nu i}$

In [ ]:
scf_eng.make_rdm1.__func__

In [ ]:
np.allclose(
    scf_eng.make_rdm1(),
    2 * scf_eng.mo_coeff[:, so] @ scf_eng.mo_coeff[:, so].T
)

In [ ]:
np.allclose(
    scf_eng.make_rdm1(),
    D
)

### 轨道能量 $\varepsilon_p$

In [ ]:
type(scf_eng.mo_energy)

In [ ]:
np.allclose(scf_eng.mo_energy, e)

### 轨道占据数

In [ ]:
type(scf_eng.mo_occ)

In [ ]:
scf_eng.mo_occ

## 参考文献